# Importando bibliotecas

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import  SMOTE
from collections import Counter
from matplotlib import pyplot as plt
import numpy as np
#Algoritmos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Importando o dataset

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/rodrigoh/aprendizado-maquina/refs/heads/main/aula03/titanic.csv')
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,"211,3375",B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,"0,9167",1,2,113781,"151,5500",C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,"151,5500",C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,"151,5500",C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,"151,5500",C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


#Verificando os tipos das colunas

**Features**

* survival - Survival (0 = No; 1 = Yes)
* class - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
* name - Name
* sex - Sex
* age - Age
* sibsp - Number of Siblings/Spouses Aboard
* parch - Number of Parents/Children Aboard
* ticket - Ticket Number
* fare - Passenger Fare
* cabin - Cabin
* embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
* boat - Lifeboat (if survived)
* body - Body number (if did not survive and body was recovered)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   object 
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   object 
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(1), int64(4), object(9)
memory usage: 143.3+ KB


In [5]:
df.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

#Preenchendo dados faltantes

In [6]:
# Garante que Age é numérica (valores inválidos viram NaN)
df['age'] = pd.to_numeric(df['age'], errors='coerce')


# Calcular a mediana de idade por grupo (Sex + Pclass)
idade_media = df.groupby(['sex', 'pclass'])['age'].median()

# Preencher valores ausentes com a mediana do grupo
def preenchimento_idade(row):
    if pd.isnull(row['age']):
        return idade_media.loc[row['sex'], row['pclass']]
    else:
        return row['age']

df['age'] = df.apply(preenchimento_idade, axis=1)

# Ajustando a coluna sexo

In [7]:
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

# Separando colunas

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   int64  
 4   age        1309 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   object 
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(2), int64(5), object(7)
memory usage: 143.3+ KB


In [9]:
X = df[['age','sex', 'pclass', 'parch', 'sibsp']]
y = df['survived']

# Seperando o dataset em treino e teste deixando 20% das instâncias para teste

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(Counter(y_train))
print(Counter(y_test))

(1047, 5)
(262, 5)
Counter({0: 665, 1: 382})
Counter({0: 144, 1: 118})


# Balanceamento dos dados

In [11]:
# Aplicando RandomUnderSampler
# sampling_strategy=0.7 → quer dizer que, no final, a classe minoritária terá 70% da quantidade da majoritária.
run = RandomUnderSampler(sampling_strategy=0.7, random_state=42)
X_run, y_run = run.fit_resample(X_train, y_train)

# Distribuição após RandomUnderSampler
print("Depois do RandomUnderSampler:", Counter(y_run))

# Aplicando o SMOTE
smote = SMOTE(random_state=42)

# Aplicando o smote
X_train, y_train = smote.fit_resample(X_run, y_run)

# Distribuição após SMOTE
print("Depois do SMOTE:", Counter(y_train))

Depois do RandomUnderSampler: Counter({0: 545, 1: 382})
Depois do SMOTE: Counter({0: 545, 1: 545})


# Padronizando os dados

In [12]:
#Criando o scaler
scaler = StandardScaler()

#Ajustando os parâmetros e aplicando nos dados de treino
X_train_scaled = scaler.fit_transform(X_train)

#Aplicando o scaler nos dados de teste
X_test_scaled = scaler.transform(X_test)

# Treinando o modelo